In [3]:
f = open("had64.txt")

H64 = []
for i in range(64):
    H64.append([1.0 if i=='+' else -1.0 for i in list(f.readline())[:-1]])
    
print(H64)

# print([[H64[i][j]-H64[j][i] for i in range(64)] for j in range(64)])

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0

In [12]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import OrderedDict

class NetworkModel(nn.Module):

    def __init__(self, inputDim):

        # Initialize the network layers.

        super(NetworkModel, self).__init__()

        self.lin1 = nn.Linear(inputDim, inputDim, bias=False)

    def forward(self, x):

        # A forward function
        # Linear function without activation

        x = self.lin1(x)
        
#         F-> WalshSpec
#         WalshSpec = F * W_t

        return x

class TrainModel():

    def __init__(self, model, device, learningRate, inputDim, batchSize, numberOfSteps, epochs):

        self.device = device
        self.net = model.to(self.device)
#         self.optimizer = optim.Adam(self.net.parameters(), lr=learningRate, weight_decay=0.00005)
        self.optimizer = optim.SGD(self.net.parameters(), lr=learningRate, momentum=0.9)
        self.inputDim = inputDim
        self.batchSize = batchSize
        self.numberOfSteps = numberOfSteps
        self.epochs = epochs
        self.H = {}
        self.H[4] = torch.tensor([[1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0], [1.0, -1.0, -1.0, 1.0]])
        self.H[8] = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0],
                            [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0],
                            [1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0],
                            [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0],
                            [1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0],
                            [1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0]])
        self.H[64] = torch.tensor(H64)
        if(self.inputDim<20):
#             For generating data of suficcient LI inputs:
            while(True):
                self.perm = torch.randperm(pow(2, self.inputDim))
#                 print("Permutation of input numbers i.e. truth tables: \n", self.perm)
                self.data = [[0.0]*(self.inputDim-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in self.perm]
                if(np.linalg.matrix_rank(self.data[:self.numberOfSteps])<self.inputDim):
                    print("Rank not large enough, generating data again")
                else:
                    print("Rank is ", np.linalg.matrix_rank(self.data[:self.numberOfSteps]))
                    break
# #             For generating one-hot data:
#             self.data = [[0.0]*i + [1.0] + [0.0]*(self.inputDim-i-1) for i in range(self.inputDim)]
            
        else:
            self.data = [[0.0 if random.random()>0.5 else 1.0 for i in range(self.inputDim)] for i in range(self.numberOfSteps)]
            print("Data generated.")
#             while(True):
#                 if(np.linalg.matrix_rank(self.data[:self.numberOfSteps])<self.inputDim):
#                     print("Rank not large enough, generating data again")
#                 else:
#                     print("Rank is ", np.linalg.matrix_rank(self.data[:self.numberOfSteps]))
#                     break
            
    
    def train(self, ):
        
        H_error = []
        for ep in range(self.epochs):
            
            for step in range(self.numberOfSteps):

                self.optimizer.zero_grad()
                input = torch.tensor(self.data[step])
                output = self.net(input)
                loss = F.mse_loss(torch.matmul(input, self.H[self.inputDim]), output)
                loss.backward()
                self.optimizer.step()
            
            weight_dict = OrderedDict(self.net.named_parameters())
            weightFunction = weight_dict['lin1.weight'].T
            H_error.append(F.mse_loss(self.H[self.inputDim], weightFunction).item())
            if (ep==0 or (ep+1)%1==0):
                print("ep= ", ep, "error:", H_error[-1], "\n", weightFunction)
                
            
        
        print("H_error array: \n", H_error)
        weight_dict = OrderedDict(self.net.named_parameters())
        weightFunction = weight_dict['lin1.weight'].T
        print("Final rounded weight function: \n", torch.round(weightFunction))



    def test(self,n):

#         test_samples = torch.tensor([[float(el) for el in row] for row in torch.randint(0, 2, (n, self.inputDim)).to(self.device)])
        test_samples = torch.tensor(self.data[self.numberOfSteps:self.numberOfSteps+n])    
        preds = self.net(test_samples)
        outputs = torch.matmul(test_samples, self.H[self.inputDim])
        print("Test:")
        for i in range(len(preds)):
            print(outputs[i], "\n", torch.round(preds[i]), "\n\n")


def main():

    # Please change the inputs here
    inputDim = 4

    # Additional inputs may not be required to change.
    learningRate = 0.02
    batchSize = 16
    numberOfSteps = 16
    epochs = 200

    # Additional Initializer
    device = torch.device('cpu')
    model =  NetworkModel(inputDim)
    trainer = TrainModel(model, device, learningRate, inputDim, batchSize, numberOfSteps, epochs)
    trainer.train()
#     trainer.test(10)
    
    print(inputDim, learningRate, batchSize, numberOfSteps, epochs)

if __name__ == '__main__':
    main()

Rank is  4
ep=  0 error: 0.6057363748550415 
 tensor([[ 0.4633,  0.1634,  0.1787, -0.1549],
        [ 0.9389, -0.0283,  0.1837, -0.0245],
        [ 1.2979,  0.3648, -0.0963,  0.0027],
        [ 0.8631, -0.3162, -0.2697,  0.1080]], grad_fn=<PermuteBackward0>)
ep=  1 error: 0.25050216913223267 
 tensor([[ 0.9072,  0.4577,  0.5121,  0.2823],
        [ 1.2144, -0.3889,  0.5140, -0.3932],
        [ 1.4631,  0.5797, -0.3712, -0.3742],
        [ 1.1635, -0.5788, -0.4978,  0.4578]], grad_fn=<PermuteBackward0>)
ep=  2 error: 0.07954470813274384 
 tensor([[ 0.8537,  0.6870,  0.7051,  0.6029],
        [ 1.0188, -0.6482,  0.7001, -0.6434],
        [ 1.1700,  0.7535, -0.6460, -0.6329],
        [ 0.9874, -0.7638, -0.7268,  0.7048]], grad_fn=<PermuteBackward0>)
ep=  3 error: 0.023929791525006294 
 tensor([[ 0.8837,  0.8331,  0.8335,  0.7841],
        [ 0.9717, -0.8044,  0.8269, -0.8084],
        [ 1.0583,  0.8668, -0.8091, -0.8033],
        [ 0.9520, -0.8717, -0.8577,  0.8384]], grad_fn=<PermuteBackw

ep=  33 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  34 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  35 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  36 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<Permute

ep=  95 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  96 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  97 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  98 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<Permute

ep=  129 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  130 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  131 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  132 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<Per

 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  186 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  187 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  188 error: 2.5157653738006047e-13 
 tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000, -1.0000,  1.0000, -1.0000],
        [ 1.0000,  1.0000, -1.0000, -1.0000],
        [ 1.0000, -1.0000, -1.0000,  1.0000]], grad_fn=<PermuteBackward0>)
ep=  189 error: 2.515765

In [55]:
import numpy as np

print(random.random())
# print(np.linalg.matrix_rank([[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0], [1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0], [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0], [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]))

0.4294725919849127
